In [49]:
#######################################################
# [1] Q-learning
#######################################################
import numpy as np

# 환경 설정 (간단한 그리드 월드 환경 가정)
n_states = 5  # 총 5개의 상태 (0~4)
n_actions = 2  # 각 상태에서 선택할 수 있는 2가지 행동 (예: 0, 1)
Q = np.zeros((n_states, n_actions))  # Q-테이블 초기화, 모든 상태-행동 값이 0으로 시작

alpha = 0.1  # 학습률: Q-값 업데이트 시 새로운 정보 반영 비율
gamma = 0.9  # 할인 계수: 미래 보상의 중요도를 조정
epsilon = 0.1  # 탐험 확률: 랜덤 행동 선택 비율

# 행동 선택 함수 (ε-greedy 정책)
def choose_action(state):
    if np.random.uniform(0, 1) < epsilon:  # ε 확률로 탐험
       return np.random.choice(n_actions)  # 랜덤으로 행동 선택
    else:  # 1-ε 확률로 활용
       return np.argmax(Q[state, :])  # Q-값이 가장 큰 행동 선택

# Q-값 업데이트 함수
def update_q(state, action, reward, next_state):
    predict = Q[state, action]  # 현재 상태-행동 값 예측
    target = reward + gamma * np.max(Q[next_state, :])  # 보상 + 다음 상태에서의 최대 Q-값
    Q[state, action] = predict + alpha * (target - predict)  # Q-값 업데이트 공식

# 학습 과정 반복
for episode in range(100):  # 100번의 학습 에피소드 실행
    state = np.random.randint(0, n_states)  # 임의의 상태에서 에피소드 시작
    while state != 4:  # 종료 상태(4)에 도달하면 에피소드 종료
        action = choose_action(state)  # 현재 상태에서 행동 선택 (탐험 또는 활용)
        next_state = np.random.randint(0, n_states)  # 랜덤으로 다음 상태로 전이
        reward = 1 if next_state == 4 else 0  # 보상 설정: 종료 상태로 전이 시 보상 1, 그 외 0
        update_q(state, action, reward, next_state)  # Q-값 업데이트
        state = next_state  # 다음 상태로 전이

# 최종 Q-테이블 출력
print(Q)  # 학습 완료된 Q-테이블 출력
	

[[0.68279676 0.29722763]
 [0.61318348 0.18622236]
 [0.64308809 0.13316469]
 [0.65223923 0.14596641]
 [0.         0.        ]]


In [69]:
#######################################################
# [2] DQN(Deep Q-Network) 
#######################################################

import numpy as np
import tensorflow as tf
from collections import deque

# 상태 및 행동 정의
n_states = 5  # 상태 공간 크기
n_actions = 2  # 행동 공간 크기

# 신경망 모델 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_dim=n_states, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(n_actions, activation='linear')
])
model.compile(optimizer='adam', loss='mse')

# Q-learning 파라미터 설정
gamma = 0.95  # 할인율
epsilon = 0.5  # 초기 탐험 확률
epsilon_min = 0.01  # 최소 탐험 확률
epsilon_decay = 0.995  # 탐험 확률 감소율
batch_size = 32  # 배치 크기
memory = deque(maxlen=2000)  # 경험 재플레이 버퍼

# 행동 선택 함수 (ε-greedy)
def choose_action(state):
    global epsilon
    if np.random.rand() <= epsilon:  # 탐험
        return np.random.choice(n_actions)
    state = np.reshape(state, [1, n_states])  # 상태를 신경망 입력 크기로 변환
    return np.argmax(model.predict(state, verbose=0))  # Q-값이 최대인 행동 선택

# 경험 재플레이 함수 (배치 학습)
def replay():
    global epsilon
    if len(memory) < batch_size:
        return  # 메모리가 충분하지 않으면 학습하지 않음
    
    # 배치 샘플링
    minibatch = np.random.choice(len(memory), batch_size, replace=False)
    states, actions, rewards, next_states, dones = [], [], [], [], []
    for i in minibatch:
        state, action, reward, next_state, done = memory[i]
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        next_states.append(next_state)
        dones.append(done)
    
    # 배열 변환
    states = np.array(states, dtype=np.float32)
    next_states = np.array(next_states, dtype=np.float32)
    rewards = np.array(rewards, dtype=np.float32)
    dones = np.array(dones, dtype=np.float32)
    
    # Q-값 업데이트
    target_q_values = model.predict(next_states, verbose=0)
    max_target_q_values = np.amax(target_q_values, axis=1)
    targets = model.predict(states, verbose=0)
    
    for i in range(batch_size):
        targets[i, actions[i]] = rewards[i] + (1 - dones[i]) * gamma * max_target_q_values[i]
    
    # 배치 학습
    model.fit(states, targets, epochs=1, verbose=0)
    
    # 탐험 확률 감소
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# 학습 반복 (에피소드 수 감소)
for episode in range(100):  # 에피소드 수 감소
    state = np.random.rand(n_states)  # 초기 상태를 임의로 설정
    done = False
    total_reward = 0
    step_count = 0
    while not done:
        action = choose_action(state)  # 행동 선택
        next_state = np.random.rand(n_states)  # 다음 상태 임의 생성
        reward = 1 if np.random.rand() > 0.5 else 0  # 보상 설정
        done = True if reward == 1 else False  # 보상이 1이면 종료
        memory.append((state, action, reward, next_state, done))  # 경험 저장
        state = next_state  # 상태 업데이트
        total_reward += reward
        step_count += 1
        replay()  # 경험 재플레이로 학습
    
    # 에피소드 결과 출력
    print(f"Episode {episode + 1}: Total Reward = {total_reward}, Steps = {step_count}, Epsilon = {epsilon:.4f}")

print("학습 완료")

# 최종 Q-값 확인 (랜덤 샘플)
test_state = np.random.rand(n_states)
test_q_values = model.predict(test_state[np.newaxis], verbose=0)
print(f"Test State: {test_state}")
print(f"Predicted Q-Values: {test_q_values}")




C:\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode 1: Total Reward = 1, Steps = 1, Epsilon = 0.5000
Episode 2: Total Reward = 1, Steps = 3, Epsilon = 0.5000
Episode 3: Total Reward = 1, Steps = 1, Epsilon = 0.5000
Episode 4: Total Reward = 1, Steps = 1, Epsilon = 0.5000
Episode 5: Total Reward = 1, Steps = 1, Epsilon = 0.5000
Episode 6: Total Reward = 1, Steps = 2, Epsilon = 0.5000
Episode 7: Total Reward = 1, Steps = 2, Epsilon = 0.5000
Episode 8: Total Reward = 1, Steps = 1, Epsilon = 0.5000
Episode 9: Total Reward = 1, Steps = 1, Epsilon = 0.5000
Episode 10: Total Reward = 1, Steps = 1, Epsilon = 0.5000
Episode 11: Total Reward = 1, Steps = 2, Epsilon = 0.5000
Episode 12: Total Reward = 1, Steps = 3, Epsilon = 0.5000
Episode 13: Total Reward = 1, Steps = 2, Epsilon = 0.5000
Episode 14: Total Reward = 1, Steps = 2, Epsilon = 0.5000
Episode 15: Total Reward = 1, Steps = 4, Epsilon = 0.5000
Episode 16: Total Reward = 1, Steps = 3, Epsilon = 0.5000
Episode 17: Total Reward = 1, Steps = 2, Epsilon = 0.4975
Episode 18: Total Rewar

In [59]:
#######################################################
# [2] DQN(Deep Q-Network) iris
#######################################################
#!pip install tensorflow
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 1. 데이터 준비
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 환경 변수
n_states = X_train.shape[1]  # 상태 차원 (Iris의 피처 수)
n_actions = len(np.unique(y))  # 행동 차원 (Iris 클래스 수)
episodes = 100  # 학습 에피소드
gamma = 0.9  # 할인 계수
epsilon = 1.0  # 초기 탐험 확률
epsilon_min = 0.1  # 최소 탐험 확률
epsilon_decay = 0.995  # 탐험 감소율
batch_size = 32  # 배치 크기
learning_rate = 0.001  # 학습률

# 2. DQN 신경망 정의
def build_model():
    model = Sequential([
        Input(shape=(n_states,)),  # Input 레이어 사용
        Dense(24, activation='relu'),
        Dense(24, activation='relu'),
        Dense(n_actions, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse')
    return model

# Q-네트워크와 타겟 네트워크 초기화
q_network = build_model()
target_network = build_model()
target_network.set_weights(q_network.get_weights())  # 가중치 동기화

# 경험 재플레이 버퍼
replay_buffer = []

# 행동 선택 함수 (ε-greedy)
def choose_action(state, epsilon):
    if np.random.rand() < epsilon:  # 탐험
        return np.random.choice(n_actions)
    else:  # 활용
        q_values = q_network.predict(state[np.newaxis], verbose=0)
        return np.argmax(q_values)

# 경험 재플레이 함수
def replay():
    if len(replay_buffer) < batch_size:
        return
    
    # 랜덤 샘플링
    minibatch = np.random.choice(range(len(replay_buffer)), batch_size, replace=False)
    minibatch = [replay_buffer[i] for i in minibatch]
    
    # 데이터 분리
    states, actions, rewards, next_states, dones = zip(*minibatch)
    
    # 배열로 변환 (데이터 일관성 유지)
    states = np.array(states, dtype=np.float32)
    next_states = np.array(next_states, dtype=np.float32)
    rewards = np.array(rewards, dtype=np.float32)
    dones = np.array(dones, dtype=np.float32)

    # Q-값 업데이트
    target_q_values = target_network.predict(next_states, verbose=0)
    max_target_q_values = np.max(target_q_values, axis=1)
    targets = q_network.predict(states, verbose=0)

    for i in range(batch_size):
        targets[i, actions[i]] = rewards[i] + (1 - dones[i]) * gamma * max_target_q_values[i]

    # 모델 학습
    q_network.fit(states, targets, epochs=1, verbose=0)

# 타겟 네트워크 업데이트 함수
def update_target_network():
    target_network.set_weights(q_network.get_weights())

# 3. DQN 학습
for episode in range(episodes):
    state_idx = np.random.randint(0, len(X_train))
    state = np.array(X_train[state_idx], dtype=np.float32)  # NumPy 배열로 변환
    done = False
    total_reward = 0

    for step in range(100):  # 최대 100 스텝 제한
        action = choose_action(state, epsilon)  # 행동 선택
        next_state_idx = np.random.randint(0, len(X_train))
        next_state = np.array(X_train[next_state_idx], dtype=np.float32)  # NumPy 배열로 변환

        # 보상 정의
        reward = 1 if y_train[next_state_idx] == action else -1
        done = (reward == 1)

        # 경험 저장 (NumPy 배열로 변환된 상태 저장)
        replay_buffer.append((state, action, reward, next_state, done))
        if len(replay_buffer) > 2000:  # 버퍼 크기 제한
            replay_buffer.pop(0)

        # 학습
        replay()
        state = next_state
        total_reward += reward
        if done:
            break

    # 탐험 감소
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # 타겟 네트워크 업데이트
    if episode % 10 == 0:
        update_target_network()

    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")

# 4. 테스트 및 평가
y_pred = []
for x in X_test:
    action = choose_action(x, epsilon=0.0)  # 테스트 시 탐험 제거
    y_pred.append(action)

accuracy = accuracy_score(y_test, y_pred)
print(f"Final Model Accuracy: {accuracy:.4f}")



Episode: 1, Total Reward: 0
Episode: 2, Total Reward: -3
Episode: 3, Total Reward: 0
Episode: 4, Total Reward: 0
Episode: 5, Total Reward: 1
Episode: 6, Total Reward: 1
Episode: 7, Total Reward: -2
Episode: 8, Total Reward: 1
Episode: 9, Total Reward: 1
Episode: 10, Total Reward: -3
Episode: 11, Total Reward: -1
Episode: 12, Total Reward: -1
Episode: 13, Total Reward: -2
Episode: 14, Total Reward: 0
Episode: 15, Total Reward: 1
Episode: 16, Total Reward: 0
Episode: 17, Total Reward: -2
Episode: 18, Total Reward: -4
Episode: 19, Total Reward: 1
Episode: 20, Total Reward: 1
Episode: 21, Total Reward: -6
Episode: 22, Total Reward: 1
Episode: 23, Total Reward: -2
Episode: 24, Total Reward: 1
Episode: 25, Total Reward: 0
Episode: 26, Total Reward: 0
Episode: 27, Total Reward: -1
Episode: 28, Total Reward: -2
Episode: 29, Total Reward: 1
Episode: 30, Total Reward: 1
Episode: 31, Total Reward: -4
Episode: 32, Total Reward: -4
Episode: 33, Total Reward: -13
Episode: 34, Total Reward: 0
Episode

In [75]:
#######################################################
# [3] SARSA(State-Action-Reward-State-Action)
#######################################################
import numpy as np

# SARSA 알고리즘을 위한 환경 설정
n_states = 5  # 상태의 개수
n_actions = 2  # 행동의 개수
Q = np.zeros((n_states, n_actions))  # Q-테이블 초기화
alpha = 0.1  # 학습률
gamma = 0.9  # 할인 계수
epsilon = 0.1  # 탐험 확률

# 행동 선택 함수 (ε-greedy)
def choose_action(state):
    if np.random.uniform(0, 1) < epsilon:  # 탐험
        return np.random.choice(n_actions)
    else:  # 활용
        return np.argmax(Q[state, :])

# Q-값 업데이트 함수
def update_q(state, action, reward, next_state, next_action):
    predict = Q[state, action]  # 현재 상태-행동의 Q-값
    target = reward + gamma * Q[next_state, next_action]  # SARSA의 타겟 값
    Q[state, action] = predict + alpha * (target - predict)  # 업데이트 공식

# 학습 반복
num_episodes = 100  # 에피소드 수
total_rewards = []  # 에피소드별 총 보상 기록
success_count = 0  # 종료 상태에 도달한 에피소드 수

for episode in range(num_episodes):
    state = np.random.randint(0, n_states)  # 초기 상태를 무작위로 설정
    action = choose_action(state)  # 초기 행동 선택
    episode_reward = 0  # 에피소드별 총 보상
    
    while state != 4:  # 종료 상태(4)에 도달하면 종료
        next_state = np.random.randint(0, n_states)  # 다음 상태 무작위 생성
        reward = 1 if next_state == 4 else 0  # 종료 상태에 도달하면 보상 1
        next_action = choose_action(next_state)  # 다음 행동 선택
        update_q(state, action, reward, next_state, next_action)  # Q-값 업데이트
        state, action = next_state, next_action  # 상태와 행동 업데이트
        episode_reward += reward  # 보상 누적
        
        if reward == 1:  # 종료 상태에 도달
            success_count += 1

    total_rewards.append(episode_reward)  # 에피소드별 총 보상 기록

# 학습 결과 평가
print("학습 완료!")
print(f"총 에피소드: {num_episodes}")
print(f"성공적으로 종료 상태에 도달한 에피소드 수: {success_count}")
print(f"성공 비율: {success_count / num_episodes:.2f}")
print(f"평균 에피소드 보상: {np.mean(total_rewards):.2f}")
print("최종 Q-테이블:")
print(Q)




학습 완료!
총 에피소드: 100
성공적으로 종료 상태에 도달한 에피소드 수: 77
성공 비율: 0.77
평균 에피소드 보상: 0.77
최종 Q-테이블:
[[0.63674193 0.25212404]
 [0.33953311 0.59365598]
 [0.59980783 0.05818173]
 [0.47228527 0.21979548]
 [0.         0.        ]]


In [23]:
#######################################################
# 스태킹(Stacking)
#######################################################
#!pip install lightgbm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 1. 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 분할 (Train, Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 2. Base Models 정의
knn = KNeighborsClassifier(n_neighbors=5)  # KNN 하이퍼파라미터 튜닝
logistic = LogisticRegression(max_iter=300)  # Logistic Regression 개선
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)  # Random Forest 개선
xgboost = xgb.XGBClassifier(n_estimators=200, learning_rate=0.1, eval_metric='mlogloss', random_state=42)  # XGBoost 개선

# 3. Base Models 학습 및 예측
# KNN
knn.fit(X_train, y_train)
knn_pred_train = knn.predict_proba(X_train)
knn_pred_test = knn.predict_proba(X_test)
knn_accuracy = accuracy_score(y_test, knn.predict(X_test))  # 정확도 계산

# Logistic Regression
logistic.fit(X_train, y_train)
logistic_pred_train = logistic.predict_proba(X_train)
logistic_pred_test = logistic.predict_proba(X_test)
logistic_accuracy = accuracy_score(y_test, logistic.predict(X_test))  # 정확도 계산

# Random Forest
rf.fit(X_train, y_train)
rf_pred_train = rf.predict_proba(X_train)
rf_pred_test = rf.predict_proba(X_test)
rf_accuracy = accuracy_score(y_test, rf.predict(X_test))  # 정확도 계산

# XGBoost
xgboost.fit(X_train, y_train)
xgb_pred_train = xgboost.predict_proba(X_train)
xgb_pred_test = xgboost.predict_proba(X_test)
xgb_accuracy = accuracy_score(y_test, xgboost.predict(X_test))  # 정확도 계산

# 4. Base Models 예측값을 하나의 데이터프레임으로 합침
# 학습 데이터
stacked_train = pd.DataFrame({
    "knn_0": knn_pred_train[:, 0], "knn_1": knn_pred_train[:, 1], "knn_2": knn_pred_train[:, 2],
    "logistic_0": logistic_pred_train[:, 0], "logistic_1": logistic_pred_train[:, 1], "logistic_2": logistic_pred_train[:, 2],
    "rf_0": rf_pred_train[:, 0], "rf_1": rf_pred_train[:, 1], "rf_2": rf_pred_train[:, 2],
    "xgb_0": xgb_pred_train[:, 0], "xgb_1": xgb_pred_train[:, 1], "xgb_2": xgb_pred_train[:, 2],
})

# 테스트 데이터
stacked_test = pd.DataFrame({
    "knn_0": knn_pred_test[:, 0], "knn_1": knn_pred_test[:, 1], "knn_2": knn_pred_test[:, 2],
    "logistic_0": logistic_pred_test[:, 0], "logistic_1": logistic_pred_test[:, 1], "logistic_2": logistic_pred_test[:, 2],
    "rf_0": rf_pred_test[:, 0], "rf_1": rf_pred_test[:, 1], "rf_2": rf_pred_test[:, 2],
    "xgb_0": xgb_pred_test[:, 0], "xgb_1": xgb_pred_test[:, 1], "xgb_2": xgb_pred_test[:, 2],
})

# 데이터 정규화 (스케일링)
scaler = StandardScaler()
stacked_train_scaled = scaler.fit_transform(stacked_train)
stacked_test_scaled = scaler.transform(stacked_test)

# 5. Final Model (LightGBM) 학습 및 예측
lgb_model = lgb.LGBMClassifier(
    random_state=42,
    min_data_in_leaf=30,        # 리프 노드 최소 데이터 수를 낮춤
    min_gain_to_split=0.1,      # 분할 수행 최소 정보 이득을 낮춤
    max_depth=7,                # 트리 최대 깊이를 늘림
    num_leaves=31,              # 리프 노드 개수를 늘림
    feature_fraction=0.9,       # 학습에 사용할 피처 비율 증가
    bagging_fraction=0.9        # 학습에 사용할 데이터 비율 증가
)

# 모델 학습
lgb_model.fit(stacked_train_scaled, y_train)

# 예측
lgb_pred = lgb_model.predict(stacked_test_scaled)

# 최종 모델 정확도
lgb_accuracy = accuracy_score(y_test, lgb_pred)

# 6. Base Model 및 최종 모델 정확도 출력
print(f"KNN Accuracy: {knn_accuracy:.4f}")
print(f"Logistic Regression Accuracy: {logistic_accuracy:.4f}")
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(f"XGBoost Accuracy: {xgb_accuracy:.4f}")
print(f"Final Model (LightGBM) Accuracy: {lgb_accuracy:.4f}")




KNN Accuracy: 0.9778
Logistic Regression Accuracy: 0.9333
Random Forest Accuracy: 0.9111
XGBoost Accuracy: 0.9333
Final Model (LightGBM) Accuracy: 0.9333


In [31]:
#######################################################
# 배깅(Bagging)
#######################################################
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

# 1. 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 분할 (Train, Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 2. Base Estimator 설정
base_estimator = DecisionTreeClassifier(max_depth=5, random_state=42)

# 3. Bagging Classifier 설정
bagging_clf = BaggingClassifier(
    estimator=base_estimator,
    n_estimators=50,  # 기본 모델 개수 증가
    max_samples=1.0,  # 전체 데이터를 사용
    max_features=1.0,  # 모든 피처를 사용
    bootstrap=True,
    random_state=42
)

# 배깅 모델 학습 및 평가
bagging_clf.fit(X_train, y_train)
bagging_pred = bagging_clf.predict(X_test)
bagging_accuracy = accuracy_score(y_test, bagging_pred)
print(f"Bagging Classifier Accuracy: {bagging_accuracy:.4f}")

# 교차 검증 (배깅)
bagging_cv_scores = cross_val_score(bagging_clf, X, y, cv=5)
print(f"Bagging Cross-Validation Accuracy: {bagging_cv_scores.mean():.4f}")

# 4. 단일 DecisionTreeClassifier 학습 및 평가
single_tree = DecisionTreeClassifier(max_depth=5, random_state=42)
single_tree.fit(X_train, y_train)
single_tree_pred = single_tree.predict(X_test)
single_tree_accuracy = accuracy_score(y_test, single_tree_pred)
print(f"Single Decision Tree Accuracy: {single_tree_accuracy:.4f}")

# 교차 검증 (단일 Decision Tree)
single_tree_cv_scores = cross_val_score(single_tree, X, y, cv=5)
print(f"Single Decision Tree Cross-Validation Accuracy: {single_tree_cv_scores.mean():.4f}")




Bagging Classifier Accuracy: 0.9333
Bagging Cross-Validation Accuracy: 0.9667
Single Decision Tree Accuracy: 0.9333
Single Decision Tree Cross-Validation Accuracy: 0.9533


In [35]:
#######################################################
# 부스팅(Boosting) - Gradient Boosting Classifier
#######################################################
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

# 1. 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 분할 (Train, Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 2. Gradient Boosting Classifier 설정 (하이퍼파라미터 조정)
gb_clf = GradientBoostingClassifier(
    n_estimators=200,      # 부스팅 스테이지 개수 증가
    learning_rate=0.05,    # 학습률 감소
    max_depth=5,           # 개별 트리의 최대 깊이 증가
    random_state=42
)

# 3. 모델 학습
gb_clf.fit(X_train, y_train)

# 4. 예측
y_pred = gb_clf.predict(X_test)

# 5. 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Gradient Boosting Classifier Accuracy: {accuracy:.4f}")

# 6. 교차 검증 (추가적인 평가)
cv_scores = cross_val_score(gb_clf, X, y, cv=5)
print(f"Cross-Validation Accuracy: {cv_scores.mean():.4f}")



Gradient Boosting Classifier Accuracy: 0.9333
Cross-Validation Accuracy: 0.9600


In [43]:
#######################################################
# 부스팅(Boosting) - xgboost
#######################################################
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 1. 데이터 로드 및 분할
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 스케일링 (선택 사항)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. XGBoost 설정 (최적화된 설정 유지)
xgb_clf = xgb.XGBClassifier(
    n_estimators=300,         # 부스팅 스테이지 증가
    learning_rate=0.1,        # 기본 학습률 사용
    max_depth=5,              # 적당한 깊이로 조정
    min_child_weight=3,       # 분할의 최소 조건 강화
    subsample=0.8,            # 샘플링 비율
    colsample_bytree=0.8,     # 피처 샘플링 비율
    random_state=42,
    use_label_encoder=False,  # 경고 제거
    eval_metric="mlogloss"    # 다중 클래스 손실 함수
)

# 3. 모델 학습
xgb_clf.fit(X_train_scaled, y_train)

# 4. 예측
y_pred = xgb_clf.predict(X_test_scaled)

# 5. 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Tuned XGBoost Classifier Accuracy: {accuracy:.4f}")

# 6. 교차 검증
cv_scores = cross_val_score(xgb_clf, X, y, cv=5)
print(f"Tuned XGBoost Cross-Validation Accuracy: {cv_scores.mean():.4f}")





C:\python\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\python\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\python\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Tuned XGBoost Classifier Accuracy: 0.9333
Tuned XGBoost Cross-Validation Accuracy: 0.9467


C:\python\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\python\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\python\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:27:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
